# Network & Cloud Security Overview

This notebook covers network security fundamentals, cloud security models, and best practices for securing infrastructure in hybrid environments.

## Topics Covered
1. Network Security Fundamentals (Firewalls, IDS/IPS, VPNs)
2. Network Segmentation
3. Cloud Shared Responsibility Model
4. IAM Best Practices
5. Security Groups & NACLs
6. Encryption (In Transit & At Rest)
7. Cloud Security Posture Management

## 1. Network Security Fundamentals

### Firewalls
Firewalls filter network traffic based on predefined rules, acting as gatekeepers between trusted and untrusted networks.

| Type | Layer | Description |
|------|-------|-------------|
| **Packet Filter** | L3-L4 | Inspects IP/port headers, stateless |
| **Stateful** | L3-L4 | Tracks connection state, allows return traffic |
| **Application (WAF)** | L7 | Deep packet inspection, understands protocols |
| **Next-Gen (NGFW)** | L3-L7 | Combines stateful + app awareness + threat intel |

```
┌─────────────────────────────────────────────────────────────────┐
│                        FIREWALL ARCHITECTURE                     │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│   INTERNET        ┌──────────────┐        INTERNAL NETWORK       │
│                   │              │                               │
│   ○ ○ ○ ○ ───────▶│   FIREWALL   │───────▶  ▣ ▣ ▣ ▣             │
│   Untrusted       │              │          Trusted              │
│   Traffic         │  ┌────────┐  │          Servers              │
│                   │  │ Rules  │  │                               │
│   ✗ Blocked       │  │ ────── │  │          ✓ Allowed            │
│   ✗ Malicious     │  │ ALLOW  │  │          ✓ Inspected          │
│                   │  │ DENY   │  │                               │
│                   │  │ LOG    │  │                               │
│                   │  └────────┘  │                               │
│                   └──────────────┘                               │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

### IDS vs IPS

```
┌─────────────────────────────────────────────────────────────────┐
│              IDS (Intrusion Detection System)                    │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│   Traffic ────┬────────────────────────────────▶ Network         │
│               │                                                  │
│               │ (copy/mirror)                                    │
│               ▼                                                  │
│          ┌─────────┐                                             │
│          │   IDS   │──────▶ ALERT (passive monitoring)          │
│          └─────────┘                                             │
│                                                                  │
├─────────────────────────────────────────────────────────────────┤
│              IPS (Intrusion Prevention System)                   │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│   Traffic ─────▶ ┌─────────┐ ─────▶ Network                     │
│                  │   IPS   │                                     │
│                  └────┬────┘                                     │
│                       │                                          │
│                       ▼                                          │
│              BLOCK + ALERT (inline, active)                      │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

**Detection Methods:**
- **Signature-based**: Matches known attack patterns (fast, misses zero-days)
- **Anomaly-based**: Detects deviations from baseline (catches novel attacks, more false positives)
- **Heuristic/Behavioral**: Analyzes behavior patterns

### VPN (Virtual Private Network)

```
┌─────────────────────────────────────────────────────────────────┐
│                      VPN TUNNEL                                  │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│   ┌──────────┐      ╔═══════════════════╗      ┌──────────┐     │
│   │  Remote  │      ║  ENCRYPTED TUNNEL ║      │ Corporate│     │
│   │  User    │◀════▶║   (IPSec/SSL)     ║◀════▶│ Network  │     │
│   └──────────┘      ╚═══════════════════╝      └──────────┘     │
│        │                    │                        │          │
│        │            PUBLIC INTERNET                  │          │
│        │         (untrusted network)                 │          │
│        │                                             │          │
│   ┌────▼─────────────────────────────────────────────▼────┐     │
│   │  Data encrypted end-to-end, invisible to attackers   │     │
│   └───────────────────────────────────────────────────────┘     │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘

VPN Types:
├── Site-to-Site: Connects entire networks (branch offices)
├── Remote Access: Individual users to corporate network
└── SSL/TLS VPN: Browser-based, no client needed
```

## 2. Network Segmentation

Network segmentation divides a network into isolated zones to limit lateral movement and contain breaches.

```
┌─────────────────────────────────────────────────────────────────────────┐
│                     NETWORK SEGMENTATION ARCHITECTURE                    │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│                              INTERNET                                    │
│                                  │                                       │
│                          ┌───────▼───────┐                               │
│                          │   PERIMETER   │                               │
│                          │   FIREWALL    │                               │
│                          └───────┬───────┘                               │
│                                  │                                       │
│    ┌─────────────────────────────┼─────────────────────────────┐         │
│    │                             │           DMZ               │         │
│    │    ┌────────┐    ┌──────────┴──────────┐    ┌────────┐    │         │
│    │    │  Web   │    │    Load Balancer    │    │  Mail  │    │         │
│    │    │ Server │    └─────────────────────┘    │ Server │    │         │
│    │    └────────┘                               └────────┘    │         │
│    └───────────────────────────┬───────────────────────────────┘         │
│                                │                                         │
│                        ┌───────▼───────┐                                 │
│                        │   INTERNAL    │                                 │
│                        │   FIREWALL    │                                 │
│                        └───────┬───────┘                                 │
│                                │                                         │
│    ┌───────────────────────────┼───────────────────────────┐             │
│    │                           │                           │             │
│    ▼                           ▼                           ▼             │
│ ┌──────────────┐    ┌──────────────────┐    ┌──────────────────┐         │
│ │   WORKSTATION│    │   APPLICATION    │    │    DATABASE      │         │
│ │   SEGMENT    │    │   SEGMENT        │    │    SEGMENT       │         │
│ │              │    │                  │    │                  │         │
│ │ ▣ ▣ ▣ ▣      │◀──▶│  ▣ App Servers   │◀──▶│  ▣ DB Servers    │         │
│ │ User PCs     │    │  ▣ API Gateways  │    │  ▣ Data Stores   │         │
│ └──────────────┘    └──────────────────┘    └──────────────────┘         │
│                                                                          │
│  ════════════════════════════════════════════════════════════════════    │
│  MICRO-SEGMENTATION (Zero Trust): Every workload isolated, verified      │
└─────────────────────────────────────────────────────────────────────────┘
```

### Segmentation Strategies

| Strategy | Description | Use Case |
|----------|-------------|----------|
| **VLAN** | Layer 2 isolation via tagged frames | Departmental separation |
| **Subnetting** | Layer 3 IP-based separation | Network topology design |
| **Firewall Zones** | Policy-based traffic control | DMZ, internal tiers |
| **Micro-segmentation** | Workload-level isolation | Zero Trust, cloud-native |
| **SDN** | Software-defined network policies | Dynamic, automated segmentation |

## 3. Cloud Shared Responsibility Model

Cloud security is a partnership between the cloud provider and customer. Responsibilities shift based on service model.

```
┌─────────────────────────────────────────────────────────────────────────┐
│              SHARED RESPONSIBILITY MODEL BY SERVICE TYPE                 │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   RESPONSIBILITY         ON-PREM    IaaS      PaaS      SaaS            │
│   ══════════════════════════════════════════════════════════════════    │
│                                                                          │
│   Data & Access          CUSTOMER   CUSTOMER  CUSTOMER  CUSTOMER        │
│   ─────────────────────────────────────────────────────────────────     │
│   Identity & Directory   CUSTOMER   CUSTOMER  CUSTOMER  SHARED          │
│   ─────────────────────────────────────────────────────────────────     │
│   Applications           CUSTOMER   CUSTOMER  CUSTOMER  PROVIDER        │
│   ─────────────────────────────────────────────────────────────────     │
│   Network Controls       CUSTOMER   CUSTOMER  SHARED    PROVIDER        │
│   ─────────────────────────────────────────────────────────────────     │
│   Operating System       CUSTOMER   CUSTOMER  PROVIDER  PROVIDER        │
│   ─────────────────────────────────────────────────────────────────     │
│   Hypervisor             CUSTOMER   PROVIDER  PROVIDER  PROVIDER        │
│   ─────────────────────────────────────────────────────────────────     │
│   Physical Infrastructure CUSTOMER  PROVIDER  PROVIDER  PROVIDER        │
│                                                                          │
│   ══════════════════════════════════════════════════════════════════    │
│   Legend:  CUSTOMER = You secure it                                      │
│            PROVIDER = Cloud provider secures it                          │
│            SHARED   = Both share responsibility                          │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### Key Takeaways

```
┌─────────────────────────────────────────────────────────────────────────┐
│                                                                          │
│   ▲  CUSTOMER RESPONSIBILITY ("Security IN the Cloud")                  │
│   │  ├── Data classification & encryption                               │
│   │  ├── Identity & access management                                   │
│   │  ├── Application security                                           │
│   │  ├── Network/firewall configuration                                 │
│   │  └── OS patching (IaaS)                                             │
│   │                                                                      │
│   ▼  PROVIDER RESPONSIBILITY ("Security OF the Cloud")                  │
│      ├── Physical data center security                                  │
│      ├── Hardware & hypervisor                                          │
│      ├── Network infrastructure                                         │
│      ├── Managed service security (PaaS/SaaS)                           │
│      └── Compliance certifications                                      │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

## 4. IAM Best Practices

Identity and Access Management (IAM) is the cornerstone of cloud security.

```
┌─────────────────────────────────────────────────────────────────────────┐
│                         IAM CORE PRINCIPLES                              │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌───────────────┐    ┌───────────────┐    ┌───────────────┐           │
│   │   IDENTITY    │    │  AUTHENTICATION│    │ AUTHORIZATION │           │
│   │               │    │               │    │               │           │
│   │  "Who are     │───▶│  "Prove it"   │───▶│  "What can    │           │
│   │   you?"       │    │               │    │   you do?"    │           │
│   │               │    │  • Password   │    │               │           │
│   │  • Users      │    │  • MFA        │    │  • Policies   │           │
│   │  • Groups     │    │  • Tokens     │    │  • Roles      │           │
│   │  • Services   │    │  • Certs      │    │  • Permissions│           │
│   └───────────────┘    └───────────────┘    └───────────────┘           │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### IAM Best Practices Checklist

| Practice | Description | Priority |
|----------|-------------|----------|
| **Least Privilege** | Grant minimum permissions needed | 🔴 Critical |
| **MFA Everywhere** | Enforce for all users, especially admins | 🔴 Critical |
| **No Root/Admin Keys** | Use IAM roles, never hardcode credentials | 🔴 Critical |
| **Role-Based Access** | Assign permissions via roles, not users | 🟡 High |
| **Regular Audits** | Review and rotate credentials periodically | 🟡 High |
| **Federation/SSO** | Centralize identity with IdP (SAML, OIDC) | 🟡 High |
| **Service Accounts** | Use dedicated identities for automation | 🟢 Medium |
| **Attribute-Based (ABAC)** | Dynamic policies based on tags/attributes | 🟢 Medium |

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    PRINCIPLE OF LEAST PRIVILEGE                          │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ❌ BAD: Overly permissive                                              │
│   ┌─────────────────────────────────────────────────────────────────┐   │
│   │  User: developer                                                │   │
│   │  Policy: AdministratorAccess  (Full access to everything!)      │   │
│   └─────────────────────────────────────────────────────────────────┘   │
│                                                                          │
│   ✅ GOOD: Scoped permissions                                            │
│   ┌─────────────────────────────────────────────────────────────────┐   │
│   │  User: developer                                                │   │
│   │  Policy: Allow s3:GetObject, s3:PutObject                       │   │
│   │          Resource: arn:aws:s3:::my-app-bucket/*                 │   │
│   │          Condition: SourceIP in [10.0.0.0/8]                    │   │
│   └─────────────────────────────────────────────────────────────────┘   │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

## 5. Security Groups & NACLs

Cloud network security uses layered controls at different levels.

```
┌─────────────────────────────────────────────────────────────────────────┐
│              SECURITY GROUPS vs NACLs (AWS Example)                      │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│                          ┌─────────────────────────┐                     │
│                          │         VPC             │                     │
│                          │                         │                     │
│    INTERNET ────────────▶│   ┌─────────────────┐   │                     │
│                          │   │  NACL (Subnet)  │◀──│── Stateless        │
│                          │   │  ═══════════════│   │   Subnet-level     │
│                          │   │                 │   │   Numbered rules   │
│                          │   │  ┌───────────┐  │   │   Explicit deny    │
│                          │   │  │ Security  │◀─│───│── Stateful         │
│                          │   │  │  Group    │  │   │   Instance-level   │
│                          │   │  │ ═════════ │  │   │   Allow-only rules │
│                          │   │  │           │  │   │   Implicit deny    │
│                          │   │  │  ┌─────┐  │  │   │                     │
│                          │   │  │  │ EC2 │  │  │   │                     │
│                          │   │  │  └─────┘  │  │   │                     │
│                          │   │  └───────────┘  │   │                     │
│                          │   └─────────────────┘   │                     │
│                          └─────────────────────────┘                     │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### Comparison Table

| Feature | Security Group | NACL |
|---------|---------------|------|
| **Scope** | Instance/ENI level | Subnet level |
| **State** | Stateful (return traffic auto-allowed) | Stateless (must allow both directions) |
| **Rules** | Allow only | Allow and Deny |
| **Evaluation** | All rules evaluated | Rules processed in order |
| **Default** | Deny all inbound, allow all outbound | Allow all (default NACL) |
| **Use Case** | Fine-grained instance control | Subnet-wide blocking |

```
┌─────────────────────────────────────────────────────────────────────────┐
│                  SECURITY GROUP RULE EXAMPLE                             │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│  Inbound Rules (Web Server):                                             │
│  ┌──────────┬──────────┬─────────────────┬──────────────────────────┐   │
│  │ Type     │ Port     │ Source          │ Description              │   │
│  ├──────────┼──────────┼─────────────────┼──────────────────────────┤   │
│  │ HTTPS    │ 443      │ 0.0.0.0/0       │ Public web traffic       │   │
│  │ HTTP     │ 80       │ 0.0.0.0/0       │ Redirect to HTTPS        │   │
│  │ SSH      │ 22       │ 10.0.0.0/8      │ Internal admin access    │   │
│  │ Custom   │ 8080     │ sg-app-servers  │ Health checks from ALB   │   │
│  └──────────┴──────────┴─────────────────┴──────────────────────────┘   │
│                                                                          │
│  💡 Best Practice: Reference security groups, not IP ranges when         │
│     communicating between cloud resources.                               │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

## 6. Encryption: In Transit & At Rest

Encryption protects data confidentiality and integrity throughout its lifecycle.

```
┌─────────────────────────────────────────────────────────────────────────┐
│                     DATA ENCRYPTION LIFECYCLE                            │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌─────────────────────────────────────────────────────────────────┐   │
│   │                    IN TRANSIT (Data in Motion)                  │   │
│   │                                                                 │   │
│   │   Client ══════════════════════════════════════════▶ Server    │   │
│   │           TLS 1.2/1.3  │  HTTPS  │  mTLS  │  VPN              │   │
│   │                                                                 │   │
│   │   Protects against:                                             │   │
│   │   • Eavesdropping (MITM)                                        │   │
│   │   • Packet sniffing                                             │   │
│   │   • Data tampering                                              │   │
│   └─────────────────────────────────────────────────────────────────┘   │
│                                                                          │
│   ┌─────────────────────────────────────────────────────────────────┐   │
│   │                    AT REST (Data in Storage)                    │   │
│   │                                                                 │   │
│   │   ┌─────────┐    ┌─────────┐    ┌─────────┐    ┌─────────┐     │   │
│   │   │   S3    │    │   EBS   │    │   RDS   │    │ Secrets │     │   │
│   │   │ 🔒 AES  │    │ 🔒 AES  │    │ 🔒 TDE  │    │ 🔒 KMS  │     │   │
│   │   └─────────┘    └─────────┘    └─────────┘    └─────────┘     │   │
│   │                                                                 │   │
│   │   Protects against:                                             │   │
│   │   • Physical theft                                              │   │
│   │   • Unauthorized disk access                                    │   │
│   │   • Data breaches                                               │   │
│   └─────────────────────────────────────────────────────────────────┘   │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### Key Management Hierarchy

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    ENCRYPTION KEY HIERARCHY                              │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│                    ┌─────────────────────────┐                           │
│                    │   Master Key (CMK/KEK)  │  ◀── HSM-protected       │
│                    │   Never leaves KMS      │      Audit logged         │
│                    └───────────┬─────────────┘                           │
│                                │                                         │
│                    ┌───────────▼─────────────┐                           │
│                    │   Data Encryption Keys  │  ◀── Encrypted by CMK    │
│                    │   (DEKs)                │      Stored with data     │
│                    └───────────┬─────────────┘                           │
│                                │                                         │
│         ┌──────────────────────┼──────────────────────┐                  │
│         ▼                      ▼                      ▼                  │
│   ┌──────────┐          ┌──────────┐          ┌──────────┐               │
│   │ S3 Data  │          │ EBS Vol  │          │ RDS DB   │               │
│   │ 🔐       │          │ 🔐       │          │ 🔐       │               │
│   └──────────┘          └──────────┘          └──────────┘               │
│                                                                          │
│   Envelope Encryption: DEK encrypts data, CMK encrypts DEK               │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

| Encryption Type | Algorithm | Key Size | Use Case |
|-----------------|-----------|----------|----------|
| **TLS 1.3** | ECDHE + AES-GCM | 256-bit | HTTPS, API calls |
| **AES-256** | Symmetric | 256-bit | Data at rest |
| **RSA** | Asymmetric | 2048/4096-bit | Key exchange, signatures |
| **ECDSA** | Asymmetric | 256-bit | Modern signatures |

## 7. Cloud Security Posture Management (CSPM)

CSPM continuously monitors cloud infrastructure for misconfigurations, compliance violations, and security risks.

```
┌─────────────────────────────────────────────────────────────────────────┐
│                        CSPM ARCHITECTURE                                 │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌─────────────────────────────────────────────────────────────────┐   │
│   │                    CLOUD ENVIRONMENTS                           │   │
│   │   ┌─────────┐    ┌─────────┐    ┌─────────┐    ┌─────────┐     │   │
│   │   │   AWS   │    │  Azure  │    │   GCP   │    │  K8s    │     │   │
│   │   └────┬────┘    └────┬────┘    └────┬────┘    └────┬────┘     │   │
│   └────────┼──────────────┼──────────────┼──────────────┼───────────┘   │
│            │              │              │              │               │
│            └──────────────┴──────┬───────┴──────────────┘               │
│                                  │                                       │
│                                  ▼                                       │
│                    ┌─────────────────────────┐                           │
│                    │       CSPM ENGINE       │                           │
│                    │  ─────────────────────  │                           │
│                    │  • Asset Inventory      │                           │
│                    │  • Config Assessment    │                           │
│                    │  • Compliance Mapping   │                           │
│                    │  • Risk Prioritization  │                           │
│                    │  • Auto-Remediation     │                           │
│                    └───────────┬─────────────┘                           │
│                                │                                         │
│         ┌──────────────────────┼──────────────────────┐                  │
│         ▼                      ▼                      ▼                  │
│   ┌──────────┐          ┌──────────┐          ┌──────────┐               │
│   │Dashboard │          │ Alerts   │          │ Reports  │               │
│   │& Metrics │          │ & SIEM   │          │Compliance│               │
│   └──────────┘          └──────────┘          └──────────┘               │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

### Common CSPM Findings & Remediation

| Finding | Risk | Remediation |
|---------|------|-------------|
| S3 bucket public | 🔴 Critical | Enable Block Public Access |
| Root account used | 🔴 Critical | Create IAM users, disable root keys |
| Unencrypted storage | 🟡 High | Enable default encryption |
| Security group 0.0.0.0/0 SSH | 🟡 High | Restrict to bastion/VPN IPs |
| MFA not enabled | 🟡 High | Enforce MFA for all IAM users |
| Logging disabled | 🟢 Medium | Enable CloudTrail, VPC Flow Logs |
| Old access keys | 🟢 Medium | Rotate keys < 90 days |

### Security Frameworks & Compliance

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    COMPLIANCE FRAMEWORKS                                 │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   ┌────────────┐  ┌────────────┐  ┌────────────┐  ┌────────────┐        │
│   │   CIS      │  │  SOC 2     │  │   HIPAA    │  │  PCI-DSS   │        │
│   │ Benchmarks │  │  Type II   │  │            │  │            │        │
│   │            │  │            │  │            │  │            │        │
│   │ Cloud best │  │ Security   │  │ Healthcare │  │ Payment    │        │
│   │ practices  │  │ controls   │  │ data       │  │ card data  │        │
│   └────────────┘  └────────────┘  └────────────┘  └────────────┘        │
│                                                                          │
│   ┌────────────┐  ┌────────────┐  ┌────────────┐  ┌────────────┐        │
│   │  ISO 27001 │  │   GDPR     │  │ NIST CSF   │  │  FedRAMP   │        │
│   │            │  │            │  │            │  │            │        │
│   │ InfoSec    │  │ EU data    │  │ Cybersec   │  │ US Gov     │        │
│   │ management │  │ privacy    │  │ framework  │  │ cloud      │        │
│   └────────────┘  └────────────┘  └────────────┘  └────────────┘        │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

## Quick Reference: Security Controls Cheat Sheet

```
┌─────────────────────────────────────────────────────────────────────────┐
│                  NETWORK & CLOUD SECURITY CHECKLIST                      │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│  NETWORK SECURITY                     CLOUD SECURITY                     │
│  ════════════════                     ══════════════                     │
│  □ Deploy NGFW at perimeter           □ Understand shared responsibility │
│  □ Enable IDS/IPS monitoring          □ Enable MFA for all users         │
│  □ Segment networks (VLAN/Subnet)     □ Apply least privilege IAM        │
│  □ Use VPN for remote access          □ Encrypt data in transit/rest     │
│  □ Implement Zero Trust               □ Use security groups properly     │
│  □ Log and monitor traffic            □ Enable CSPM/security tooling     │
│  □ Regular vulnerability scans        □ Audit logs and CloudTrail        │
│  □ Keep systems patched               □ Implement guard rails/policies   │
│                                                                          │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│  DEFENSE IN DEPTH LAYERS                                                 │
│  ═══════════════════════                                                 │
│                                                                          │
│    ┌─────────────────────────────────────────────────────────────┐      │
│    │ 1. PERIMETER    │ Firewall, WAF, DDoS protection            │      │
│    ├─────────────────┼───────────────────────────────────────────┤      │
│    │ 2. NETWORK      │ Segmentation, IDS/IPS, VPN                │      │
│    ├─────────────────┼───────────────────────────────────────────┤      │
│    │ 3. IDENTITY     │ IAM, MFA, SSO, RBAC                       │      │
│    ├─────────────────┼───────────────────────────────────────────┤      │
│    │ 4. COMPUTE      │ Hardened OS, patching, EDR                │      │
│    ├─────────────────┼───────────────────────────────────────────┤      │
│    │ 5. APPLICATION  │ Secure code, input validation, SAST      │      │
│    ├─────────────────┼───────────────────────────────────────────┤      │
│    │ 6. DATA         │ Encryption, DLP, classification          │      │
│    └─────────────────┴───────────────────────────────────────────┘      │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

# Network & Cloud Security

- Segmentation
- WAF/DDoS
- Zero Trust
